# Advent of Code 2024 Day 18 

### Setup

In [1]:
from aocd import get_data, submit

day = 18
year = 2024


In [ ]:
with open('example.txt', 'r') as file:
    raw_sample_data = "".join(file.readlines())

raw_sample_data[:100]

In [ ]:
raw_test_data = get_data(day=day, year=year)

raw_test_data[:]

##### Data Parsing

In [ ]:
from typing import Tuple
import numpy as np

def parse_data(raw_data:str, shape:Tuple[int, int]):
    lines = raw_data.splitlines()
    arr = np.full(shape, '.')
    
    # note inverted x, y due to x being the distanec from the left edge
    bytes_arr = np.array([(int(y), int(x)) for line in lines for x, y in [line.split(',')]])

    return {
        'memory': arr,
        'bytes': bytes_arr
    }


sample_data = parse_data(raw_sample_data, (7, 7))
test_data = parse_data(raw_test_data, (71, 71))

sample_data

### Part One!

In [24]:
use_sample_data = False
part = 'a'

In [ ]:
data = sample_data if use_sample_data else test_data

data

In [26]:
from typing import Literal, Union, List

Wall = Literal['#']
Open = Literal['.']

Tile = Union[Wall, Open]

In [27]:
from functools import total_ordering

@total_ordering
class KeyList(object):
    def __init__(self, key, lst:List):
        self.key = key
        self.val = lst

    def __lt__(self, other):
        return self.key < other.key

    def __eq__(self, other):
        return self.key == other.key
    
    def __iter__(self):
        return iter((self.key, self.val))
    
    def __getitem__(self, idx):
        return self.val[idx]
    
    def __setitem__(self, idx, value):
        self.val[idx] = value

    def extend(self, key, value:List):
        val = self.val[:] + value
        return KeyList(key, val)

In [28]:
def fill_bytes(memory:np.ndarray, bytes:np.ndarray):
    mem = memory.copy()
    unique_bytes = np.unique(bytes, axis=0)
    
    rows = unique_bytes[:, 0]
    cols = unique_bytes[:, 1]

    mem[rows, cols] = '#'

    return mem

In [29]:
def get_candidate_positions(array:np.ndarray, pos:np.ndarray):
    # move up, down, left, right
    moves = np.array([
        [-1, 0],
        [1, 0],
        [0, -1],
        [0, 1]
    ]) + pos

    # validate that none are out of bounds
    rows = moves[:, 0]
    cols = moves[:, 1]

    mask = (rows >= 0) & (rows < array.shape[0]) & (cols >= 0) & (cols < array.shape[1])
    moves = moves[mask]

    # validate that none are tile blockers (#)
    rows = moves[:, 0]
    cols = moves[:, 1]

    return moves[array[rows, cols] != '#']


In [30]:
def euclidean_distance(a, b):
    return np.sqrt(np.sum((a - b) ** 2))

def manhattan_distance(a, b):
    return np.sum(np.abs(a - b))

def dummy_heuristic(a, b):
    return 0

In [31]:
def cost(a, b):
    return 1

In [32]:
import heapq

def get_escape_path(array:np.ndarray, start, end, cost_fn=cost, heuristic_fn=dummy_heuristic):
    queue = [KeyList(0, [start])]
    visited = set()

    while queue:
        steps, path = heapq.heappop(queue)
        pos = path[-1]

        if pos[0] == end[0] and pos[1] == end[1]:
            return path

        if tuple(pos) in visited:
            continue

        visited.add(tuple(pos))

        for candidate in get_candidate_positions(array, pos):
            if tuple(candidate) in visited:
                continue

            new_path = path[:] + [candidate]
            new_steps = steps + cost_fn(pos, candidate) + heuristic_fn(candidate, end)
            heapq.heappush(queue, KeyList(new_steps, new_path))
    
    return None

In [33]:
def plot_esacpe_path(array:np.ndarray, path):
    mem = array.copy()

    for pos in path:
        mem[pos[0], pos[1]] = 'O'

    return mem

In [ ]:
fill_count = 12 if use_sample_data else 1024
bytes = data['bytes'][:fill_count]

filled_memory = fill_bytes(data['memory'].copy(), bytes.copy())

start = np.array([0, 0])
end = np.array([ filled_memory.shape[0] - 1, filled_memory.shape[1] - 1])

path = get_escape_path(filled_memory, start, end)

part_a_answer = len(path) - 1
part_a_answer

In [ ]:
if not use_sample_data and part == 'a':
    submit(answer=part_a_answer, part='a', day=day, year=year, reopen=True)

### Part Two!

In [65]:
use_sample_data = False
part='b'

In [ ]:
data = sample_data if use_sample_data else test_data

len(data['bytes'])

In [ ]:
bytes = data['bytes'].copy()
memory = data['memory'].copy()
start = np.array([0, 0])
end = np.array([ memory.shape[0] - 1, memory.shape[1] - 1])

for i in range(len(bytes)):
    print(i, bytes[i])
    filled_memory = fill_bytes(memory.copy(), bytes[:i + 1])

    path = get_escape_path(filled_memory, start, end)

    if not path:
        part_b_answer = ",".join([ str(x) for x in bytes[i][::-1] ])
        break

part_b_answer

In [ ]:
if not use_sample_data and part == 'b':
    submit(answer=part_b_answer, part='b', day=day, year=year, reopen=True)